In [1]:
import os
os.environ['PADRE_ROOT'] = '/group/bgross/work/padre'

In [2]:
import padre as p
sub_list_fates = sorted(p.subjects(tags='FATES'))

In [3]:
fates_anat_list = []
fates_rest_list = []
for sub in sub_list_fates:
    sub_anat_list = [str(d) for d in sub.dsets('anatomy')]
    sub_rest_list = [str(d) for d in sub.dsets('rest')]
    if len(sub_rest_list)!=0:
        fates_anat_list.append(sub_anat_list)
        fates_rest_list.append(sub_rest_list)
    #print([str(d) for d in sub.dsets('rest')])

In [4]:
import shutil

def copy_file(source_path, destination_path):
    """
    Copies a file from source_path to destination_path.

    Args:
        source_path (str): The full path of the source file.
        destination_path (str): The full path of the destination file.
    """
    prefix = ['']
    try:
        shutil.copy2(source_path, destination_path)
        print("File copied successfully from '{}' to '{}'".format(source_path, destination_path))
    except IOError as e: #IOError is the correct error to catch in python 2.7
        print("Error: {}".format(e))

In [5]:
import subprocess
import re

def get_afni_attributes(nifti_file):
    """
    Retrieves AFNI attributes TAXIS_OFFSETS and TAXIS_FLOATS from a NIFTI file.

    Args:
        nifti_file (str): Path to the NIFTI file (.nii.gz).

    Returns:
        dict: A dictionary containing 'SliceTiming' (from TAXIS_OFFSETS)
              and 'RepetitionTime' (extracted from TAXIS_FLOATS), or None
              if the information cannot be extracted.
    """
    try:
        command = "3dAttribute -all {}".format(nifti_file)
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        if process.returncode != 0:
            print("Error running 3dAttribute: {}".format(stderr))
            return None
        output = stdout
    except Exception as e:
        print("An error occurred: {}".format(e))
        return None

    attributes = {}

    # Extract TAXIS_OFFSETS for SliceTiming
    slice_timing_match = re.search(r"TAXIS_OFFSETS = (.+?)(?:\n|$)", output)
    if slice_timing_match:
        slice_timing_str = slice_timing_match.group(1).strip()
        attributes['SliceTiming'] = [float(x) for x in slice_timing_str.split()]

    # Extract TAXIS_FLOATS for RepetitionTime (assuming it's the second value)
    repetition_time_match = re.search(r"TAXIS_FLOATS = (.+?)(?:\n|$)", output)
    if repetition_time_match:
        taxis_floats_str = repetition_time_match.group(1).strip()
        taxis_floats = [float(x) for x in taxis_floats_str.split()]
        if len(taxis_floats) >= 2:
            attributes['RepetitionTime'] = taxis_floats[1]

    return attributes

In [6]:
import nibabel as nib
import json

def create_json(nifti_file, orig_fname, task_name="rest"):
    # Collect slice timing and repetition time information from AFNI 3dAttribute
    afni_attr = get_afni_attributes(nifti_file)
    
    # Create a BIDS-compliant JSON structure
    json_data = {
        "SliceTiming": afni_attr['SliceTiming'],
        "RepetitionTime": afni_attr['RepetitionTime'],
        "OriginalFilename": orig_fname,
        "TaskName": task_name
    }

    # Generate output JSON filename
    json_fname = nifti_file.replace("_bold.nii.gz", "_bold.json")

    # Save JSON file
    with open(json_fname, "w") as json_file:
        json.dump(json_data, json_file, indent=4)

    print('JSON file created: '+json_fname)

In [7]:
import nibabel as nib

for sub_indx in range(len(fates_rest_list)):
    for rest_indx in range(len(fates_rest_list[sub_indx])):
        sub_label = str(sub_indx+1).zfill(2)
        ses_label = str(rest_indx+1).zfill(2)
        
        # Create directory in BIDS format and copy original file to the new directory
        new_func_dir = "bids_data/sub-{}/ses-{}/func/".format(sub_label, ses_label)
        rest_dir, rest_fname = os.path.split(fates_rest_list[sub_indx][rest_indx])
        
        if not os.path.exists(new_func_dir):
            os.makedirs(new_func_dir)
        copy_file(fates_rest_list[sub_indx][rest_indx],new_func_dir)
        
        # Rename file to satisfy BIDS conventional filename
        cur_fname = new_func_dir+rest_fname
        dirs = new_func_dir.split('/')
        bids_fname = new_func_dir+dirs[1]+'_'+dirs[2]+'_task-rest_bold.nii.gz'
        os.rename(cur_fname, bids_fname)       
        
        # Create a json dataset to comply with BIDS format
        create_json(bids_fname,rest_fname)
        
        for anat_indx in range(len(fates_anat_list[sub_indx])):
            anat_dir, anat_fname = os.path.split(fates_anat_list[sub_indx][anat_indx])
            if anat_dir==rest_dir:
                new_anat_dir = "bids_data/sub-{}/ses-{}/anat/".format(sub_label, ses_label)
                if not os.path.exists(new_anat_dir):
                    os.makedirs(new_anat_dir)
 
                copy_file(fates_anat_list[sub_indx][anat_indx],new_anat_dir)
                
                # Rename file to satisfy BIDS conventional filename
                cur_fname = new_anat_dir+anat_fname
                dirs = new_anat_dir.split('/')
                bids_fname = new_anat_dir+dirs[1]+'_'+dirs[2]+'_T1w.nii.gz'
                os.rename(cur_fname, bids_fname)
                # Create a json file to store the original file
                json_anat_data = {
                    "MagneticFieldStrength": 3,
                    "OriginalFilename": anat_fname
                }
                # Generate output JSON filename
                json_fname = bids_fname.replace(".nii.gz", ".json")
                # Save JSON file
                with open(json_fname, "w") as json_file:
                    json.dump(json_anat_data, json_file, indent=4)

File copied successfully from '/group/bgross/work/padre/Data/6173/sessions/20111116/09303237-20111116-144448-REST_2-6072_images.nii.gz' to 'bids_data/sub-01/ses-01/func/'
JSON file created: bids_data/sub-01/ses-01/func/sub-01_ses-01_task-rest_bold.json
File copied successfully from '/group/bgross/work/padre/Data/6173/sessions/20111116/09303237-20111116-133748-AX_3D-190_images.nii.gz' to 'bids_data/sub-01/ses-01/anat/'
File copied successfully from '/group/bgross/work/padre/Data/6173/sessions/20111116/09303237-20111116-143708-REST_1-6072_images.nii.gz' to 'bids_data/sub-01/ses-02/func/'
JSON file created: bids_data/sub-01/ses-02/func/sub-01_ses-02_task-rest_bold.json
File copied successfully from '/group/bgross/work/padre/Data/6173/sessions/20111116/09303237-20111116-133748-AX_3D-190_images.nii.gz' to 'bids_data/sub-01/ses-02/anat/'
File copied successfully from '/group/bgross/work/padre/Data/6848/sessions/05112012/ep.6848-05112012_103000-rest1.nii.gz' to 'bids_data/sub-02/ses-01/func/'

In [8]:
json_data_des = {
    "Name": "FATES resting state functional connectivity analysis",
    "BIDSVersion": "1.8.0",
    "DatasetTypes": "raw"                  
}
# Generate output JSON filename
json_fname = "bids_data/dataset_description.json"
# Save JSON file
with open(json_fname, "w") as json_file:
    json.dump(json_data_des, json_file, indent=4)